In [1]:
import tensorflow as tf
print(tf.__version__)

tf.enable_eager_execution()

1.14.0


In [2]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

W0818 15:20:27.899250 140507400566656 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0818 15:20:48.923701 140507400566656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


W0818 15:21:33.853514 140507400566656 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_label_final = np.array(training_labels)
testing_label_final = np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_len = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_len, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len)

In [11]:
reverse_word_index = dict([(value, key) for key, value in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b i saw this film on true movies which automatically made me <OOV> but actually it was good why not because of the amazing plot twists or breathtaking dialogue of which there is little but because actually despite what people say i thought the film was accurate in it's depiction of teenagers dealing with <OOV> br br it's not <OOV> creek they're not <OOV> cool witty characters who <OOV> through sexuality with <OOV> knowledge they're kids and they act like kids would br br they're blunt awkward and annoyingly confused about everything yes this could be by accident and they could just be bad actors but i don't think so <OOV> <OOV> gives when not trying to be cool a
b"I saw this film on True Movies (which automatically made me sceptical) but actually - it was good. Why? Not because of the amazing plot twists or breathtaking dialogue (of which there is little) but because actually, despite what people say I thought the film was accurate in it's depiction of teenagers dealing with pregnancy.

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 10
model.fit(padded, training_label_final, epochs = num_epochs, validation_data = (testing_padded, testing_label_final))

W0818 17:01:58.058448 140507400566656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 269us/sample - loss: 0.4776 - acc: 0.7592 - val_loss: 0.3420 - val_acc: 0.8510
Epoch 2/10
25000/25000 [==============================] - 5s 217us/sample - loss: 0.2402 - acc: 0.9073 - val_loss: 0.3684 - val_acc: 0.8396
Epoch 3/10
25000/25000 [==============================] - 5s 218us/sample - loss: 0.0980 - acc: 0.9746 - val_loss: 0.4523 - val_acc: 0.8267
Epoch 4/10
25000/25000 [==============================] - 5s 219us/sample - loss: 0.0251 - acc: 0.9968 - val_loss: 0.5258 - val_acc: 0.8249
Epoch 5/10
25000/25000 [==============================] - 5s 219us/sample - loss: 0.0066 - acc: 0.9993 - val_loss: 0.5941 - val_acc: 0.8250
Epoch 6/10
25000/25000 [==============================] - 5s 215us/sample - loss: 0.0021 - acc: 0.9999 - val_loss: 0.6577 - val_acc: 0.8232
Epoch 7/10
25000/25000 [==============================] - 5s 216us/sample - loss: 9.3056e-04 - acc: 1.0000 - v

In [16]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding = 'utf-8')
out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_v.write(word + '\n')
    out_m.write('\t'.join([str(x) for x in embeddings]) + "\n")
    
out_v.close()
out_m.close()

In [0]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')